## Import Libraries

In [1]:
import googlemaps
import pandas as pd
import time
import os

## Read Config File

In [2]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
input_file = config['DEFAULT']['Input-File']
output_file = config['DEFAULT']['Output-File']
threshold = config['DEFAULT']['Threshold']
api_key = config['DEFAULT']['Google-API-Key']
address_column = config['DEFAULT']['Address-Column']

## Read CSV/Excel File

In [7]:
if os.path.splitext(input_file)[1] == ".csv":
    df = pd.read_csv(input_file, header = None, encoding="UTF-8")
else:
    df = pd.read_excel(input_file, header = None, encoding="UTF-8")

In [8]:
print(len(df))

62


## Check Threshold (Default Value : 1000)

In [9]:
if int(threshold) > 1000:
    df = df.iloc[1000:int(threshold),]
else:
    df = df.iloc[0:1000,]

## Get Address Value

In [11]:
if address_column.isdigit():
    address_list = df[int(address_column)].tolist()
else:
    address_list = df[address_column].tolist()

## Add New Columns (Latitude, Longtitude)

In [12]:
new_column_list = ["latitude","longtitude"]

In [13]:
for i in new_column_list:
    df[i] = None

## Connect Google Geocoding API

In [14]:
gmaps = googlemaps.Client(key=api_key)

## Append Value into New Columns

In [15]:
start = time.time()

for index,row in df.iterrows():
    address = df.loc[index,0]
    try:
        geocode_result = gmaps.geocode(address)
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        long = geocode_result[0]["geometry"]["location"]["lng"]
        coordinate = [lat,long]
        df.loc[[index],new_column_list] = coordinate
    except:
        print("error address: " + address)
        
end = time.time()
print(end-start)

29.38843274116516


## Write into CSV

In [16]:
df.rename(columns={0: "address"}, inplace=True)
df.to_csv(output_file,index=0,encoding='UTF-8')